In [1]:
!pip install -U spacy==2.3.5

In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

In [4]:
def is_float(n):
    try:
        support_float_with_norwegian_format = n.replace(',','.')
        float_n = float(support_float_with_norwegian_format)
    except ValueError:
        return False
    else:
        return True
    
# test
print(is_float('test'))
print(is_float('4'))
print(is_float('4.5'))
print(is_float('4,5'))

False
True
True
True


In [5]:
def is_int(n):
    try:
        float_n = float(n)
        int_n = int(float_n)
    except ValueError:
        return False
    else:
        return float_n == int_n

# test
print(is_int('test'))
print(is_int('4'))
print(is_int('4.5'))

False
True
False


In [6]:
text = "Radioutstyret på skip og flyttbare innretninger skal ha landbasert vedlikehold i samsvar med produsentens instruksjoner. bankfiske, bankfiske I, bankfiske II, havfiske, havfiske I, havfiske II, fjordfiske, isfarvann I, isfarvann II, kystfiske, fartsområde 1, fartsområde 2, fartsområde 3, fartsområde 4, fartsområde 5, europeisk fart, stor kystfart, internasjonal reise, intent sertifisert fartsområde, nord- og østersjøfart, oversjøiskfart, kort internasjonal reise, fart på innsjøer og elver, uinnskrenket fart."

In [7]:
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher

# Blank model.
# Does not have a POS tagger.
nlp = English()
doc = nlp(text)
matcher = Matcher(nlp.vocab)

#
# START - spaCy patterns
#

matcher.add(
    "TRADE_AREA",
    [
        [
            {"LOWER": {"IN": ["bankfiske"]}}
        ],
        [
            {"LOWER": {"IN": ["havfiske"]}}
        ],
        [
            {"LOWER": {"IN": ["fjordfiske"]}}
        ],
        [
            {"LOWER": {"IN": ["isfarvann"]}}
        ],
        [
            {"LOWER": {"IN": ["kystfiske"]}}
        ],
        [
            {"LOWER": {"IN": ["fartsområde"]}}
        ],
        [
            {"LOWER": {"IN": ["europeisk"]}},
            {"LOWER": {"IN": ["fart"]}}
        ],
        [
            {"LOWER": {"IN": ["stor"]}},
            {"LOWER": {"IN": ["kystfart"]}}
        ],
        [
            {"LOWER": {"IN": ["internasjonal"]}},
            {"LOWER": {"IN": ["reise"]}}
        ],
        [
            {"LOWER": {"IN": ["intent"]}},
            {"LOWER": {"IN": ["sertifisert"]}}
        ],
        [
            {"LOWER": {"IN": ["nord-"]}},
            {"LOWER": {"IN": ["og"]}},
            {"LOWER": {"IN": ["østersjøfart"]}}
        ],
        [
            {"LOWER": {"IN": ["oversjøiskfart"]}}
        ],
        [
            {"LOWER": {"IN": ["fart"]}},
            {"LOWER": {"IN": ["på"]}},
            {"LOWER": {"IN": ["innsjøer"]}},
            {"LOWER": {"IN": ["og"]}},
            {"LOWER": {"IN": ["elver"]}}
        ],
        [
            {"LOWER": {"IN": ["uinnskrenket"]}},
            {"LOWER": {"IN": ["fart"]}}
        ]
    ])

#
# END - spaCy patterns
#

result = []

for match_id, token_start, token_end in matcher(doc):

    match_id_as_string = nlp.vocab.strings[match_id]
    final_token_start = token_start
    final_token_end = token_end

    spacy_pattern_detection = doc[token_start:token_end]
    spacy_pattern_detection_as_lower_text = spacy_pattern_detection.text.lower()
    
    #
    # Expand VESSEL_TYPE?
    #

    if match_id_as_string == "TRADE_AREA" and token_start > 0:

        if spacy_pattern_detection_as_lower_text == "internasjonal reise":

            prev_word_1_token_number = token_start - 1
            prev_word_1_token = doc[prev_word_1_token_number]

            if prev_word_1_token.text.lower() == "kort":
                # Example: kort internasjonal reise
                # Expanding.
                final_token_start = prev_word_1_token_number

        elif spacy_pattern_detection_as_lower_text == "fartsområde":

            next_word_1_token_number = token_end
            next_word_1_token = doc[next_word_1_token_number]

            if is_int(next_word_1_token.text.lower()):
                # Example: fartsområde 5
                # Expanding.
                final_token_end = next_word_1_token_number + 1

        elif (spacy_pattern_detection_as_lower_text == "bankfiske" or 
            spacy_pattern_detection_as_lower_text == "havfiske" or
            spacy_pattern_detection_as_lower_text == "isfarvann"):

            next_word_1_token_number = token_end
            next_word_1_token = doc[next_word_1_token_number]

            if (next_word_1_token.text.lower() == "i" or 
                next_word_1_token.text.lower() == "ii"):
                # Example: havfiske II
                # Expanding.
                final_token_end = next_word_1_token_number + 1

    #
    # convert token_span to char_span.
    # char_span is needed to display correctly withdisplacy.render().
    #
    span = doc[final_token_start:final_token_end]
    span_char_start = span[0].idx
    span_char_end = span[-1].idx + len(span[-1].text)

    # return result
    identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
    result.append(identified_entity)

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"TRADE_AREA": "darksalmon"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)

# Functions run from .py file

In [8]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_TRADE_AREA_no

from spacy_matching_rule_identify_TRADE_AREA_no import identify_TRADE_AREA_in_text

importlib.reload(spacy_matching_rule_identify_TRADE_AREA_no)

################
################

identify_TRADE_AREA_in_text(text)

[{'start': 121, 'end': 130, 'label': 'TRADE_AREA'},
 {'start': 132, 'end': 143, 'label': 'TRADE_AREA'},
 {'start': 145, 'end': 157, 'label': 'TRADE_AREA'},
 {'start': 159, 'end': 167, 'label': 'TRADE_AREA'},
 {'start': 169, 'end': 179, 'label': 'TRADE_AREA'},
 {'start': 181, 'end': 192, 'label': 'TRADE_AREA'},
 {'start': 194, 'end': 204, 'label': 'TRADE_AREA'},
 {'start': 206, 'end': 217, 'label': 'TRADE_AREA'},
 {'start': 219, 'end': 231, 'label': 'TRADE_AREA'},
 {'start': 233, 'end': 242, 'label': 'TRADE_AREA'},
 {'start': 244, 'end': 257, 'label': 'TRADE_AREA'},
 {'start': 259, 'end': 272, 'label': 'TRADE_AREA'},
 {'start': 274, 'end': 287, 'label': 'TRADE_AREA'},
 {'start': 289, 'end': 302, 'label': 'TRADE_AREA'},
 {'start': 304, 'end': 317, 'label': 'TRADE_AREA'},
 {'start': 319, 'end': 333, 'label': 'TRADE_AREA'},
 {'start': 335, 'end': 348, 'label': 'TRADE_AREA'},
 {'start': 350, 'end': 369, 'label': 'TRADE_AREA'},
 {'start': 371, 'end': 389, 'label': 'TRADE_AREA'},
 {'start': 3